In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
url = "https://raw.githubusercontent.com/Vincent2308/Corporate-Bankruptcy/refs/heads/main/data/clean/taiwan/taiwan.csv"

df= pd.read_csv(url)
df.head()

In [ ]:
df.info()
df['Bankrupt?'].value_counts(normalize=True)

### Firm Size

We generate some values usiung the ratios available to try and approximate some values which relate to the size of the firm.

In [ ]:
df["size_retained"] = df[" Retained Earnings to Total Assets"]
df["size_working_capital"] = df[" Working Capital to Total Assets"]
df["size_networth"] = df[" Net worth/Assets"]
df["size_current_liab"] = df[" Current Liability to Assets"]
df["size_op"] = df[" Operating Expense Rate"]


df["size_index"]= (df["size_retained"] + df["size_working_capital"] + df["size_networth"] +df["size_current_liab"]+df["size_op"] )
# we add 1.0001 to ensure the log is greater than 0 since that is undefined.
# We decided that 1.0001 was large enough to ensure it was positive to actualyl make a differece
# but not too large so that it would distort our resutls
df["log_size"] = np.log(df["size_index"]+ 1.0001)
 

### Multicolinearity

We remove highly correlated features.

In [ ]:
f = plt.figure(figsize=(10, 10))
plt.matshow(df.corr(), fignum=f.number)
plt.xticks(range(df.select_dtypes(['number']).shape[1]), df.select_dtypes(['number']).columns, fontsize=5, rotation=90)
plt.yticks(range(df.select_dtypes(['number']).shape[1]), df.select_dtypes(['number']).columns, fontsize=5)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
plt.title('Correlation Matrix', fontsize=16)
plt.show()

In [ ]:
def remove_correlated(df, threshold=0.90):
    corr = df.corr().abs()

    # I used GPT for this part
    # it calculates the upper triangle above the diagonal since that has a corr of 1.
    # this avoids us calculating each correlation twice, since the correlation between A&B = corr B&A.
    upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))

    drop_cols = [c for c in upper.columns if any(upper[c] > threshold)]
    return df.drop(columns=drop_cols), drop_cols


In [ ]:
df_reduced, dropped_columns = remove_correlated(df.drop(columns=["Bankrupt?"]))
df_reduced["Bankrupt?"] = df["Bankrupt?"]

df_reduced.head()


In [ ]:
f = plt.figure(figsize=(10, 10))
plt.matshow(df_reduced.corr(), fignum=f.number)
plt.xticks(range(df_reduced.select_dtypes(['number']).shape[1]), df_reduced.select_dtypes(['number']).columns, fontsize=5, rotation=90)
plt.yticks(range(df_reduced.select_dtypes(['number']).shape[1]), df_reduced.select_dtypes(['number']).columns, fontsize=5)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
plt.title('Correlation')

### Train-Test Split

In [ ]:
from sklearn.model_selection import train_test_split

x = df_reduced.drop(columns=["Bankrupt?"])
y = df_reduced['Bankrupt?']

x_train, x_test, y_train, y_test =  train_test_split(x, y, test_size = 0.2, stratify =y, random_state = 42)

### Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x_train_sc = scaler.fit_transform(x_train)
x_test_sc  = scaler.transform(x_test)

### SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 42)
x_train_sc_sm, y_train_sm, = smote.fit_resample(x_train_sc, y_train)

### X.X Early-Warning Indicators
We could maybe add some more ratios, but they will probably have similar variances.

In [ ]:
ratios=[" Current Liability to Assets"," Net Income to Total Assets"," Working Capital to Total Assets"," Retained Earnings to Total Assets"]
variance_df = df.groupby("Bankrupt?")[ratios].var()
variance_df


### Model Evaluation Function

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, RocCurveDisplay

def evaluate_model(model, x_test_sc, y_test):
    y_pred = model.predict(x_test_sc)
    y_prob = model.predict_proba(x_test_sc)[:, 1]

    acc = accuracy_score(y_test, y_pred)
    pre = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)

    print(f'Accuracy: {acc:.3f}')
    print(f'Precision: {pre:.3f}')
    print(f'Recall: {rec:.3f}')
    print(f'F1-Score: {f1:.3f}')
    print(f'AUC Score: {auc:.3f}')

    RocCurveDisplay.from_estimator(model, x_test_sc, y_test)
    plt.show()

## Train Models

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

### Logistic Regression, without SMOTE

In [ ]:
lr = LogisticRegression(max_iter=500, class_weight="balanced")
lr.fit(x_train_sc, y_train)
evaluate_model(lr, x_test_sc, y_test)

### Logistic Regression + SMOTE

In [ ]:
lr_sm = LogisticRegression(max_iter=500)
lr_sm.fit(x_train_sc_sm, y_train_sm)
evaluate_model(lr_sm, x_test_sc, y_test)

### Logistic Regrssion PCA

In [ ]:
pca = PCA(n_components=1)
x_train_pca = pca.fit_transform(x_train_sc)
x_test_pca = pca.transform(x_test_sc)

lr_pca = LogisticRegression()
lr_pca.fit(x_train_pca, y_train)
evaluate_model(lr_pca, x_test_pca, y_test)

### Logistic Regrssion PCA + SMOTE

In [ ]:
x_train_pca_sm = pca.fit_transform(x_train_sc_sm)
lr_pca_sm = LogisticRegression()
lr_pca_sm.fit(x_train_pca_sm, y_train_sm)
evaluate_model(lr_pca_sm, x_test_pca, y_test)

### Random Forest, without SMOTE


In [ ]:
rf = RandomForestClassifier(n_estimators=500, class_weight="balanced", random_state=42)
rf.fit(x_train_sc, y_train)
evaluate_model(rf, x_test_sc, y_test)

### Random Forest + SMOTE


In [ ]:
rf_sm = RandomForestClassifier(n_estimators=500, random_state=42)
rf_sm.fit(x_train_sc_sm, y_train_sm)
evaluate_model(rf_sm, x_test_sc, y_test)

### Random forrest LDA without SMOTE

In [ ]:
lda = LinearDiscriminantAnalysis(n_components=1)
X_train_lda = lda.fit_transform(x_train_sc, y_train)
X_test_lda = lda.transform(x_test_sc)

rf_lda = RandomForestClassifier(n_estimators=1000, random_state=42, n_jobs=-1)
rf_lda.fit(X_train_lda, y_train)
evaluate_model(rf_lda, X_test_lda, y_test)

### Random forrest LDA + SMOTE

In [ ]:
X_train_lda_sm = lda.fit_transform(x_train_sc_sm, y_train_sm)
rf_lda_sm = RandomForestClassifier(n_estimators=1000, random_state=42, n_jobs=-1)
rf_lda_sm.fit(X_train_lda_sm, y_train_sm)
evaluate_model(rf_lda_sm, X_test_lda, y_test)

### K-Nearest Neighbors, without SMOTE

In [ ]:
knn = {}
for k in [3, 99]:
    model = KNeighborsClassifier(n_neighbors=k, weights='distance', n_jobs=-1)
    model.fit(x_train_sc, y_train)
    knn[str(k)] = model
    evaluate_model(knn[str(k)], x_test_sc, y_test)

### K-Nearest Neighbors + SMOTE

In [ ]:
knn_sm = {}
for k in [3, 99]:
    model = KNeighborsClassifier(n_neighbors=k, weights='distance', n_jobs=-1)
    model.fit(x_train_sc_sm, y_train_sm)
    knn_sm[str(k)] = model
    evaluate_model(knn_sm[str(k)], x_test_sc, y_test)

### XGBoost, without SMOTE


In [ ]:
xgb = XGBClassifier(eval_metric="logloss", random_state=42)
xgb.fit(x_train_sc, y_train)
evaluate_model(xgb, x_test_sc, y_test)

### XGBoost + SMOTE


In [ ]:
xgb_sm = XGBClassifier(eval_metric="logloss", random_state=42)
xgb_sm.fit(x_train_sc_sm, y_train_sm)
evaluate_model(xgb_sm, x_test_sc, y_test)

### Feature Importance

In [ ]:
coefficients = lr.coef_[0]
odds_ratios = np.exp(coefficients)

coefficients_df = pd.DataFrame({
'Feature' : x.columns,
"Coefficients" : coefficients,
"Odds ratio" : odds_ratios
})

print(coefficients_df.sort_values(by="Coefficients", ascending=False))
coefficients_df.head(15)

fi_rf = pd.Series(rf.feature_importances_, index=x.columns).sort_values(ascending=False)
fi_rf.head(15)

fi_xgb = pd.Series(xgb.feature_importances_,index=x.columns).sort_values(ascending=False)
fi_xgb.head(15)

## Model Comparisons

In [ ]:
reduced_results = []

def add_result(name, model, data):
    # evaluate the mdodel based on the previoulsy chosen metrics
    y_pred = model.predict(data)

    # gets the predicted probability the the sample belongs to bankrupt = 1
    y_prob = model.predict_proba(data)[:,1]

    reduced_results.append({"Model": name,"Auc": roc_auc_score(y_test, y_prob),
        "Recall": recall_score(y_test, y_pred),"Precision": precision_score(y_test, y_pred),
        "F1": f1_score(y_test, y_pred)
    })

add_result("Logistic regression", lr, x_test_sc)
add_result("Logistic regression (SMOTE)", lr_sm, x_test_sc)

add_result("RandomForest", rf, x_test_sc)
add_result("RandomForest (SMOTE)", rf_sm, x_test_sc)

add_result("XGBoost", xgb, x_test_sc)
add_result("XGBoost (SMOTE)", xgb_sm, x_test_sc)

add_result("KNN k=3", knn_sm["3"], x_test_sc)
add_result("KNN k=3 (SMOTE)", knn_sm["3"], x_test_sc)

add_result("KNN k=99", knn["99"], x_test_sc)
add_result("KNN k=99 (SMOTE)", knn_sm["99"], x_test_sc)

add_result("Logistic regression PCA", lr_pca, X_test_lda)
add_result("Logistic regression PCA (SMOTE)", lr_pca_sm, X_test_lda)

add_result("Random forrest LDA", rf_lda, X_test_lda)
add_result("Random forrest LDA (SMOTE)", rf_lda_sm, X_test_lda)


In [ ]:
metrics_table = pd.DataFrame(reduced_results)
metrics_table_display = metrics_table[['Model', 'Auc', 'Recall', 'Precision', 'F1']]
metrics_table_display.set_index('Model', inplace=True)

plt.figure(figsize=(8, 4))
plt.axis('off')
plt.title("Model Metrics Table", fontsize=16, pad=20)
from pandas.plotting import table
table(plt.gca(), metrics_table_display, loc='center', colWidths=[0.2]*len(metrics_table_display.columns))
plt.show()



### Models Across Metrics


In [ ]:
import seaborn as sns
metrics_melted = metrics_table.melt(id_vars="Model", value_vars=["Auc", "Recall", "Precision", "F1"], 
                                    var_name="Metric", value_name="Value")

plt.figure(figsize=(12, 6))
sns.barplot(x="Model", y="Value", hue="Metric", data=metrics_melted)
plt.xticks(rotation=45, ha='right')
plt.ylim(0, 1)
plt.title("Comparison of Models Across Metrics", fontsize=16)
plt.ylabel("Score")
plt.legend(title="Metric")
plt.tight_layout()
plt.show()

### ROC and Precision-Recall curves

In [ ]:
from sklearn.metrics import RocCurveDisplay, PrecisionRecallDisplay

# --- Define models and labels ---
models = {
    "Logistic Regression": lr,
    "Logistic Regression (SMOTE)": lr_sm,
    "Random Forest": rf,
    "Random Forest (SMOTE)": rf_sm,
    "XGBoost": xgb,
    "XGBoost (SMOTE)": xgb_sm,
    "KNN k=3": knn["3"],
    "KNN k=3 (SMOTE)": knn_sm["3"]
}

# --- ROC Curves ---
plt.figure(figsize=(10, 8))
for name, model in models.items():
    y_prob = model.predict_proba(x_test_sc)[:,1]
    RocCurveDisplay.from_predictions(y_test, y_prob, name=name, ax=plt.gca())

plt.title("ROC Curves for All Models")
plt.grid(True)
plt.legend(loc='lower right', fontsize=10)
plt.show()

# --- Precision-Recall Curves ---
plt.figure(figsize=(10, 8))
for name, model in models.items():
    y_prob = model.predict_proba(x_test_sc)[:,1]
    PrecisionRecallDisplay.from_predictions(y_test, y_prob, name=name, ax=plt.gca())

plt.title("Precision-Recall Curves for All Models")
plt.grid(True)
plt.legend(loc='lower right', fontsize=10)
plt.show()


### Feature importance

In [ ]:
# Logistic Regression (absolute coefficients)
coef = pd.Series(np.abs(lr.coef_[0]), index=x.columns).sort_values(ascending=False).head(15)
plt.figure(figsize=(8,6))
sns.barplot(x=coef.values, y=coef.index)
plt.title("Top 15 Feature Importances - Logistic Regression")
plt.xlabel("Absolute Coefficient")
plt.ylabel("Feature")
plt.show()

# Random Forest
rf_importances = pd.Series(rf.feature_importances_, index=x.columns).sort_values(ascending=False).head(15)
plt.figure(figsize=(8,6))
sns.barplot(x=rf_importances.values, y=rf_importances.index)
plt.title("Top 15 Feature Importances - Random Forest")
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.show()

# XGBoost
xgb_importances = pd.Series(xgb.feature_importances_, index=x.columns).sort_values(ascending=False).head(15)
plt.figure(figsize=(8,6))
sns.barplot(x=xgb_importances.values, y=xgb_importances.index)
plt.title("Top 15 Feature Importances - XGBoost")
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.show()




# SMOTE Effect
 

In [ ]:
smote_metrics = pd.DataFrame({
    "Model": ["Logistic Regression", "Random Forest", "XGBoost", "KNN k=3"],
    "Non-SMOTE AUC": [
        roc_auc_score(y_test, lr.predict_proba(x_test_sc)[:,1]),
        roc_auc_score(y_test, rf.predict_proba(x_test_sc)[:,1]),
        roc_auc_score(y_test, xgb.predict_proba(x_test_sc)[:,1]),
        roc_auc_score(y_test, knn3.predict_proba(x_test_sc)[:,1])
    ],
    "SMOTE AUC": [
        roc_auc_score(y_test, lr_sm.predict_proba(x_test_sc)[:,1]),
        roc_auc_score(y_test, rf_sm.predict_proba(x_test_sc)[:,1]),
        roc_auc_score(y_test, xgb_sm.predict_proba(x_test_sc)[:,1]),
        roc_auc_score(y_test, knn3_sm.predict_proba(x_test_sc)[:,1])
    ]
})

smote_metrics.set_index("Model", inplace=True)
smote_metrics.plot(kind="bar", figsize=(8,5))
plt.title("AUC: SMOTE vs Non-SMOTE")
plt.ylabel("AUC Score")
plt.show()

# Collect predicted probabilities
pred_probs = pd.DataFrame({
    "LR": lr.predict_proba(x_test_sc)[:,1],
    "LR_SMOTE": lr_sm.predict_proba(x_test_sc)[:,1],
    "RF": rf.predict_proba(x_test_sc)[:,1],
    "RF_SMOTE": rf_sm.predict_proba(x_test_sc)[:,1],
    "XGB": xgb.predict_proba(x_test_sc)[:,1],
    "XGB_SMOTE": xgb_sm.predict_proba(x_test_sc)[:,1],
    "KNN": knn["3"].predict_proba(x_test_sc)[:,1],
    "KNN_SMOTE": knn_sm["3"].predict_proba(x_test_sc)[:,1],
})

plt.figure(figsize=(10,8))
sns.heatmap(pred_probs.corr(), annot=True, cmap="coolwarm", vmin=0, vmax=1)
plt.title("Correlation of Predicted Probabilities Across Models")
plt.show()

